In [ ]:
#!pip install dash 
#!pip install dash_bootstrap_components
#!pip install plotly
import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo
from dash.dependencies import Input, Output
from dash import dash_table

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP],
                title='Superstore Exploratory Data Analysis',
                update_title='Loading...')
#Loading the data set
df = pd.read_csv('Superstore sales data.csv')
#drop Country column
df = df.drop(['Country'], axis=1)

#Changing the data types of following columns 
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date']  = pd.to_datetime(df['Ship Date'])

new_df = df

# Create a Quarter Column
df['Quarter'] = pd.PeriodIndex(df['Order Date'], freq = 'Q')
df['Quarter'] = df['Quarter'].astype(str)

#Defining required colors
colors=['#1C4E80','#EA6A47','#A5D8DD','#7E909A','#F1F1F1','#202020','#0091D5']
selected_colors = colors[:3] 

#Defining some required variables at the future stages
sales_by_region_segment = df.groupby(['Region', 'Segment'])['Sales'].sum().reset_index()
grouped_data = df.groupby(['Segment','Order Date']).sum()['Sales'].reset_index()
sales_by_quarter_segment = df.groupby(['Quarter', 'Segment']).count()['Sales'].unstack()
sales_by_region_category = df.groupby(['Region', 'Category'])['Sales'].sum().reset_index()

# Region wise Sales
data_top_10_region = new_df.groupby("Region")[["Sales"]].sum().reset_index().sort_values(by="Sales",
                                                                                                   ascending=False)
# Top 10 State Sales
data_top_10_states = new_df.groupby("State")[["Sales"]].sum().reset_index().sort_values(by="Sales",
                                                                                                  ascending=False)
# Top 10 City Sales
data_top_10_cities = new_df.groupby("City")[[ "Sales"]].sum().reset_index().sort_values(by="Sales",
                                                                                                 ascending=False)
# Top 10 Sub-category wise Sales
data_top_10_sub_categories = new_df.groupby("Sub-Category")[["Sales"]].sum().reset_index().sort_values(
    by="Sales", ascending=False)

#Top_bar
navbar = dbc.Navbar(
    [
      html.Div([
          html.H2("Superstore Dashboard", className="display-5",style={"color": "white", "font-size": "28px"}),
         
          html.P("Developed By Group - 09",  className="lead",style={"color": "white", "font-size": "18px"}),   
      ],
        #style={"display": "flex", "align-items": "center"},
         # className="ml-auto",
        ),
        dbc.NavbarToggler(id="navbar-toggler-1"),
    ],
    color='#1C4E80',
    sticky="top",
    dark=True,
)

# Main Window
# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

second_navbar = dbc.Navbar(
    [html.Div(
        [
        dbc.Row(
             [
                 dbc.Col(
        dbc.Nav(
            [
                dbc.NavLink("Dataset", href="/dataset", id="page-2-link", style={"color": "#1C4E80", "font-size": "20px"}),
                dbc.NavLink("Product Level Analysis", href="/product_level_analysis", id="page-5-link", style={"color": "#1C4E80", "font-size": "20px"}),
                dbc.NavLink("Segment Analysis", href="/segment_analysis", id="page-6-link", style={"color": "#1C4E80", "font-size": "20px"}),
                dbc.NavLink("Ship Mode Analysis", href="/ship_mode_analysis", id="page-7-link", style={"color": "#1C4E80", "font-size": "20px"}),
                dbc.NavLink("Top 10 Sales", href="/top_10", id="page-8-link", style={"color": "#1C4E80", "font-size": "20px"}),
            ],
            className="ml-auto",
            pills=True,
            style={"fontSize": "18px"}
             
            ),md=12,),
        ],),
        ],
    
    #className="mb-3",
    #style=SIDEBAR_STYLE,
    ),
     #className="container",
     dbc.NavbarToggler(id="navbar-toggler-2"),
    ] , 
    color='#CED2CC',
    
)
content = html.Div(id="page-content", style=CONTENT_STYLE)
app.layout = html.Div([navbar, dcc.Location(id="url"), second_navbar, content])


@app.callback(
    [Output(f"page-{i}-link", "active") for i in range(1, 9)],
    [Input("url", "pathname")],
)
def toggle_active_links(pathname):
    if pathname == "/":
        # Treat page 1 as the homepage / index
        return True, False, False, False, False
    return [pathname == f"/page-{i}" for i in range(1, 9)]

@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)

def render_page_content(pathname):
    if pathname == "/":
        return html.Div([html.H3("Welcome to Superstore Dashboard"),
                         html.Img(src='/assets/img/img.jpg', height="500px"),
                         html.Div([html.H4("Exploratory Data Analysis"),
                                   html.P("Exploratory Data Analysis (EDA) refers to the critical process of "
                                          "performing initial investigations on data so as to discover patterns,"
                                          "to spot anomalies,to test hypothesis and to check assumptions with the "
                                          "help of summary statistics and graphical representations."),
                                   ]),
                         ])
    elif pathname == "/dataset":
        return html.Div([html.H4("Superstore Dataset"),
                         html.P([
                            # "The dataset is available ",
                            # html.A("here", href="https://bit.ly/3i4rbWl", target="_blank"),
                             html.P(
                                 "The SampleSuperstore dataset "
                                 "consists of 9994 records with 13 columns as 'Ship Mode', 'Segment', 'Country', "
                                 "'City', 'State', 'Postal Code', 'Region', 'Category', 'Sub-Category', 'Sales'. ")
                         ]),
                         dash_table.DataTable(
                             id='table',
                             columns=[{"name": i, "id": i} for i in df.columns],
                             data=df.to_dict('records'),
                         )])
    elif pathname == "/product_level_analysis":
        return dbc.Container(fluid=True,
                children=[html.Div([html.H4("Product Level Analysis"),
                         dbc.Row(
                             [
                                 dbc.Col(dcc.Graph(
                                     id='bar',
                                     figure={
                                         'data': [go.Pie(
                                             labels=new_df['Category'].value_counts().index.values,
                                             values=new_df['Category'].value_counts().values,
                                             marker=dict(colors=colors),
                                             hoverinfo='label+value',
                                            textinfo='label+percent',
                                            #textfont=dict(size=13),
                                            hole=.7,
                                            rotation=45
                                         )],
                                         'layout': go.Layout(
                                              hovermode='closest',
                                             title={
                                                'text': "Unique Categories of the Products" ,

                                                'xanchor': 'center',
                                                'yanchor': 'top'},

                                            legend={
                                                'orientation': 'h',
                                                'bgcolor': 'white',
                                                'xanchor': 'center', },
                                            font=dict(
                                                family="sans-serif",
                                                #size=12,
                                                color='black')
                                         ),
                                     }
                                 ),md=6,),
                                
                                 dbc.Col(dcc.Graph(
                                     id='bar',
                                     figure={
                                       'data': [go.Bar(
                                           x=sales_by_region_category[sales_by_region_category['Category'] == category]['Region'],
                                           y=sales_by_region_category[sales_by_region_category['Category'] == category]['Sales'],
                                           name=category,
                                           marker={'color': colors[i]}
                                       )
                                        for i, category in enumerate(sales_by_region_category['Category'].unique())
                                               ],
                                         'layout':go.Layout(
                                             title='Total Sales by Region and Category',
                                             xaxis={'title': 'Region'},
                                            yaxis={'title': 'Total Sales'},
                                             barmode='stack'
                                         ),
                                     }
                                 ),md=6,),
                             ],),
                         dbc.Row(
                                 dbc.Col(dcc.Graph(
                                 id='bar',
                                 figure={
                                     'data': [go.Bar(
                                         x=new_df['Sub-Category'],
                                         y=new_df[new_df['Region'] == 'East'].value_counts().values,
                                         name="East",
                                         marker=dict(color=colors[2])
                                     ), go.Bar(
                                         x=new_df['Sub-Category'],
                                         y=new_df[new_df['Region'] == 'West'].value_counts().values,
                                         name="West",
                                         marker=dict(color=colors[1])
                                     ), go.Bar(
                                         x=new_df['Sub-Category'],
                                         y=new_df[new_df['Region'] == 'Central'].value_counts().values,
                                         name="Central",
                                         marker=dict(color=colors[3])
                                     ), go.Bar(
                                         x=new_df['Sub-Category'],
                                         y=new_df[new_df['Region'] == 'South'].value_counts().values,
                                         name="South",
                                         marker=dict(color="#CED2CC")
                                     )],
                                     'layout': go.Layout(
                                         title="Count of Sub-Category region wise",
                                         xaxis=dict(ticks=''),
                                         yaxis=dict(ticks=''),
                                         #width=900, height=600,
                                         barmode="group"
                                     ),
                                 }
                             ),md=12,),
                         ),
                         ]),
                    ],)
    elif pathname == "/segment_analysis":
        return dbc.Container(fluid=True,
                children=[html.Div([html.H4("Segment Analysis"),
                         dbc.Row(
                             [
                                 dbc.Col(dcc.Graph(
                                     id='bar',
                                     figure={
                                         'data': [go.Pie(
                                             labels=new_df['Segment'].value_counts().index.values,
                                             values=new_df['Segment'].value_counts().values,
                                             marker=dict(colors=colors),
                                             hoverinfo='label+value',
                                            textinfo='label+percent',
                                            #textfont=dict(size=13),
                                            hole=.7,
                                            rotation=45)],
                                         'layout': go.Layout(
                                             hovermode='closest',
                                             title={
                                                'text': "Customer Segment Distribution " ,

                                                'xanchor': 'center',
                                                'yanchor': 'top'},

                                            legend={
                                                'orientation': 'h',
                                                'bgcolor': 'white',
                                                'xanchor': 'center', },
                                            font=dict(
                                                family="sans-serif",
                                                #size=12,
                                                color='black')
                                         ),
                                     }
                                 ),md=6,),
                                 dbc.Col(dcc.Graph(
                                     id='bar',
                                     figure={
                                       'data': [go.Bar(
                                           x=sales_by_region_segment[sales_by_region_segment['Segment'] == segment]['Region'],
                                           y=sales_by_region_segment[sales_by_region_segment['Segment'] == segment]['Sales'],
                                           name=segment,
                                           marker={'color': colors[i]}
                                       )
                                        for i, segment in enumerate(sales_by_region_segment['Segment'].unique())
                                               ],
                                         'layout':go.Layout(
                                             title='Total Sales by Region and Segment',
                                             xaxis={'title': 'Region'},
                                             yaxis={'title': 'Total Sales'},
                                             barmode='stack'
                                         ),
                                     }
                                 ),md=6,),
                             ]
                         ),
                          dbc.Row(
                             dbc.Col(dcc.Graph(
                                     id='bar',
                                     figure={
                                         'data': [go.Scatter(
                                             x=sales_by_quarter_segment.index,
                                             y=sales_by_quarter_segment[segment],
                                             mode='lines',
                                             name=segment,
                                             line=dict(color=color)
                                            )
                                            for segment, color in zip(sales_by_quarter_segment.columns, selected_colors)
                                        ],
                                         'layout': go.Layout(
                                            title='Quarterly Sales by Customer Segment',
                                            xaxis={'title': 'Quarter'},
                                            yaxis={'title': 'Sales Count'}
                                         ),
                                    }
                                 ),md=12,),
                                ),       
                            ]),
                        ],)
    elif pathname == "/ship_mode_analysis":
        return dbc.Container(fluid=True,
                children=[html.Div([html.H4("Ship Mode Analysis"),
                         dbc.Row(
                             [
                                 dbc.Col(dcc.Graph(
                                     id='bar',
                                     figure={
                                         'data': [go.Pie(
                                             labels=new_df['Ship Mode'].value_counts().index.values,
                                             values=new_df['Ship Mode'].value_counts().values,
                                             marker=dict(colors=colors),
                                             hoverinfo='label+value',
                                            textinfo='label+percent',
                                            #textfont=dict(size=13),
                                            hole=.7,
                                            rotation=45
                                         )],
                                         'layout': go.Layout(
                                             hovermode='closest',
                                             title={
                                                'text': "Ship Mode Distribution" ,

                                                'xanchor': 'center',
                                                'yanchor': 'top'},

                                            legend={
                                                'orientation': 'h',
                                                'bgcolor': 'white',
                                                'xanchor': 'center', },
                                            font=dict(
                                                family="sans-serif",
                                                #size=12,
                                                color='black')
                                             
                                         ),
                                     }
                                 ),md=6,),
                              
                             dbc.Col(dcc.Graph(
                                 id='bar',
                                 figure={
                                     'data': [go.Bar(
                                         x=new_df['Ship Mode'],
                                         y=new_df[new_df['Region'] == 'East'].value_counts().values,
                                         name="East",
                                         marker=dict(color=colors[0])
                                     ), go.Bar(
                                         x=new_df['Ship Mode'],
                                         y=new_df[new_df['Region'] == 'West'].value_counts().values,
                                         name="West",
                                         marker=dict(color=colors[6])
                                     ), go.Bar(
                                         x=new_df['Ship Mode'],
                                         y=new_df[new_df['Region'] == 'Central'].value_counts().values,
                                         name="Central",
                                         marker=dict(color=colors[3])
                                     ), go.Bar(
                                         x=new_df['Ship Mode'],
                                         y=new_df[new_df['Region'] == 'South'].value_counts().values,
                                         name="South",
                                         marker=dict(color="#CED2CC")
                                     )],
                                     'layout': go.Layout(
                                         title="Count of Ship modes region wise",
                                         xaxis=dict(ticks=''),
                                         yaxis=dict(ticks=''),
                                         #width=900, height=600,
                                         barmode="group"
                                     ),
                                 }
                             ),md=6,),
                          ]
                         ),
                        ]),
                    ],)
    elif pathname == "/top_10":
        return html.Div([
            dcc.Tabs([
                dcc.Tab(label='Region', children=[
                    html.Div([html.H4("Region wise Sales"), dash_table.DataTable(
                        id='table',
                        columns=[{"name": i, "id": i} for i in data_top_10_region.columns],
                        data=data_top_10_region.to_dict('records'),
                    )]),
                    html.H5("Visualization"),
                    dbc.Row([

                        dbc.Col(dcc.Graph(
                            id='bar',
                            figure={
                                'data': [go.Bar(
                                    x=data_top_10_region['Region'],
                                    y=data_top_10_region['Sales'],
                                    marker=dict(color=colors[3])

                                )],
                                'layout': go.Layout(
                                    title="Region-wise Sales",
                                    xaxis=dict(ticks=''),
                                    yaxis=dict(ticks=''),
                                    width=500, height=500,
                                ),
                            }
                        )),
                    ]),
                ]),
                dcc.Tab(label='State', children=[
                    html.Div([html.H4("Top 10 State wise  Sales"), dash_table.DataTable(
                        id='table',
                        columns=[{"name": i, "id": i} for i in data_top_10_states.columns],
                        data=data_top_10_states.head(10).to_dict('records'),
                    )]),
                    html.H5("Visualization"),
                    dbc.Row([

                        dbc.Col(dcc.Graph(
                            id='bar',
                            figure={
                                'data': [go.Bar(
                                    x=data_top_10_states['State'].head(10),
                                    y=data_top_10_states['Sales'].head(10),
                                    marker=dict(color=colors[6])

                                )],
                                'layout': go.Layout(
                                    title="State-wise Sales",
                                    xaxis=dict(ticks=''),
                                    yaxis=dict(ticks=''),
                                    width=500, height=500,
                                ),
                            }
                        )),
                    ]),
                ]),
                dcc.Tab(label='City', children=[
                    html.Div([html.H4("Top 10 City wise  Sales"), dash_table.DataTable(
                        id='table',
                        columns=[{"name": i, "id": i} for i in data_top_10_cities.columns],
                        data=data_top_10_cities.head(10).to_dict('records'),
                    )]),
                    html.H5("Visualization"),
                    dbc.Row([

                        dbc.Col(dcc.Graph(
                            id='bar',
                            figure={
                                'data': [go.Bar(
                                    x=data_top_10_cities['City'].head(10),
                                    y=data_top_10_cities['Sales'].head(10),
                                    marker=dict(color=colors[0])

                                )],
                                'layout': go.Layout(
                                    title="City-wise Sales",
                                    xaxis=dict(ticks=''),
                                    yaxis=dict(ticks=''),
                                    width=500, height=500,
                                ),
                            }
                        )),
                    ]),
                ]),
                dcc.Tab(label='Sub-Category', children=[
                    html.Div([html.H4("Top 10 Sub-category wise Sales"), dash_table.DataTable(
                        id='table',
                        columns=[{"name": i, "id": i} for i in data_top_10_sub_categories.columns],
                        data=data_top_10_sub_categories.head(10).to_dict('records'),
                    )]),
                    html.H5("Visualization"),
                    dbc.Row([

                        dbc.Col(dcc.Graph(
                            id='bar',
                            figure={
                                'data': [go.Bar(
                                    x=data_top_10_sub_categories['Sub-Category'].head(10),
                                    y=data_top_10_sub_categories['Sales'].head(10),
                                    marker=dict(color=colors[1])

                                )],
                                'layout': go.Layout(
                                    title="Sub-Category-wise Sales",
                                    xaxis=dict(ticks=''),
                                    yaxis=dict(ticks=''),
                                    width=500, height=500,
                                ),
                            }
                        )),
                    ]),
                ]),
            ])
        ])
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )


app.run_server()